# Benchmarks

## Surprise on Data

In [28]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate
import pandas as pd

In [29]:
df = pd.read_csv('data/cleaned.csv')
df.head()

,app_id,is_recommended,user_id
0,534380,1,10531
1,42700,0,185293
2,602960,1,319249
3,976730,0,747545
4,1091500,1,2113544


In [30]:
reader = Reader(line_format='item rating user', sep=',', skip_lines=1, rating_scale=(0, 1))
data = Dataset.load_from_file('data/cleaned.csv', reader=reader)
full_train = data.build_full_trainset()

In [ ]:
svd = SVD()
results = cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)

In [31]:
svd_fit = svd.fit(full_train)

In [44]:
svd_fit.bi

array([-0.07391577,  0.02504232,  0.08813493, ...,  0.07392884,
       -0.41369429,  0.04063683])

In [97]:
user = 50
user = full_train.to_raw_uid(user)
top = []
for item in full_train.all_items():
    item = full_train.to_raw_iid(item)
    prediction = svd_fit.predict(user, item)
    uid, iid, true, pred = prediction[0], prediction[1], prediction[2], prediction[3]
    top.append((pred, iid))
    top.sort(key=lambda x: x[0], reverse=True)
    top = top[:10]

In [98]:
df2 = pd.read_csv('data/games.csv')

In [99]:
print(top)

[(0.9298358152217477, '1709170'), (0.9194860750327817, '791330'), (0.9168279957662467, '1078280'), (0.9092671373791268, '1444480'), (0.9053243056672475, '302350'), (0.892891335227362, '897220'), (0.8910542571803921, '527420'), (0.8906115514323233, '1545450'), (0.8810546624802352, '1352930'), (0.8808289450284936, '2051420')]


In [100]:
df2[df2['app_id'].isin([int(id) for r, id in top])]

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
95,302350,Tex Murphy: Under a Killing Moon,2014-06-12,True,True,True,Very Positive,96,93,9.99,9.99,0.0,True
117,527420,Demon Hunter 3: Revelation,2016-12-01,True,True,True,Very Positive,88,148,9.99,9.99,0.0,True
2080,1078280,Ruins Seeker,2020-05-28,True,False,False,Very Positive,93,553,14.99,19.99,25.0,True
3939,1545450,Incredibox,2021-04-30,True,True,False,Overwhelmingly Positive,97,1871,4.99,4.99,0.0,True
5639,1444480,Turing Complete,2021-10-02,True,True,True,Very Positive,94,1206,19.99,19.99,0.0,True
15621,1709170,Paradise Marsh,2022-10-13,True,True,False,Very Positive,98,177,9.99,9.99,0.0,True
17001,2051420,Beltex,2022-07-08,True,False,False,Very Positive,91,100,4.99,4.99,0.0,True
18104,791330,Fallen ~Makina and the City of Ruins~,2018-02-23,True,False,False,Very Positive,85,1553,3.74,14.99,75.0,True
18853,897220,Summer Pockets,2020-02-05,True,False,False,Overwhelmingly Positive,98,2015,44.99,44.99,0.0,True
19223,1352930,HAAK,2022-08-24,True,False,False,Very Positive,89,1570,17.99,17.99,0.0,True


## Custom Algorithm on MovieLens